In [1]:
import pandas as pd
import numpy as np
from catboost import CatBoostClassifier, Pool
from sklearn.metrics import roc_curve, auc, log_loss

/home/ubuntu/.software/anaonda3/lib/python3.6/importlib/_bootstrap.py:205: RuntimeWarning: compiletime version 3.5 of module '_catboost' does not match runtime version 3.6
  return f(*args, **kwds)


In [2]:
data = pd.read_csv('../data/catboost/data/train_data.csv')
train_data = data[data['eval_set'] != 'test']
valid_data = data[data['eval_set'] == 'test']

In [3]:
train_features = np.array(train_data.iloc[:,3:])
train_label = train_data['reordered'].values
cat_features = [0,1,2,3,4]

In [4]:
valid_features = np.array(valid_data.iloc[:,3:])
valid_label = valid_data['reordered'].values
cat_features = [0,1,2,3,4]

In [5]:
params = {}
params['iterations'] = 50
params['learning_rate'] = 0.03
params['depth'] = 6
params['random_seed'] = 108
params['thread_count'] = 7
params['auto_stop_pval'] = 10

In [6]:
model = CatBoostClassifier(**params)

In [7]:
model.fit(train_features, train_label, cat_features, verbose=True)

Borders generated
0:	learn 0.6643654948passed: 12.5 sec	total: 20.1s	remaining: 16m 23s
1:	learn 0.6378139829passed: 11.8 sec	total: 31.8s	remaining: 12m 44s
2:	learn 0.6130992592passed: 10.4 sec	total: 42.2s	remaining: 11m 1s
3:	learn 0.5904114135passed: 9.07 sec	total: 51.3s	remaining: 9m 49s
4:	learn 0.5695103152passed: 10.6 sec	total: 1m 1s	remaining: 9m 17s
5:	learn 0.5498353759passed: 10.5 sec	total: 1m 12s	remaining: 8m 51s
6:	learn 0.5316690748passed: 9.74 sec	total: 1m 22s	remaining: 8m 24s
7:	learn 0.5153317626passed: 10.1 sec	total: 1m 32s	remaining: 8m 4s
8:	learn 0.5003660109passed: 9.41 sec	total: 1m 41s	remaining: 7m 43s
9:	learn 0.4861329128passed: 9.7 sec	total: 1m 51s	remaining: 7m 25s
10:	learn 0.4730264301passed: 10.6 sec	total: 2m 1s	remaining: 7m 12s
11:	learn 0.4607463623passed: 9.75 sec	total: 2m 11s	remaining: 6m 57s
12:	learn 0.4498254161passed: 11.5 sec	total: 2m 23s	remaining: 6m 47s
13:	learn 0.4397043574passed: 10.6 sec	total: 2m 33s	remaining: 6m 35s
14:	

In [8]:
predictions = model.predict_proba(valid_features)

In [9]:
fpr, tpr, thresh = roc_curve(valid_label, predictions[:,1])
2*auc(fpr,tpr) - 1

0.40596999563743741

In [10]:
data = pd.read_csv('../data/catboost/data/score_data.csv')
features = np.array(data.iloc[:,3:])
label = data['reordered'].values

In [11]:
predictions = model.predict_proba(features)

In [12]:
fpr, tpr, thresh = roc_curve(label, predictions[:,1])
2*auc(fpr,tpr) - 1

0.38296142475877715

In [13]:
driver = data[['user_id','product_id']]
driver['catboost'] = predictions[:,1]

/home/ubuntu/.software/anaonda3/lib/python3.6/site-packages/ipykernel_launcher.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  


In [14]:
driver.to_csv('../data/catboost/catboost.csv', index=False)